In [59]:
import pdb

import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# SVM classifier, logistic regression, naivebayes classifier, xgboost classifer, RF classifier, OneVRest
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load in our Data

In [3]:
with open('../data/interim/text_target.pkl', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    text_target = pickle.load(f)

# Train test split

In [ ]:
X = text_target.cleaned_text
y = text_target.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Run Vectorization on Texts 

In [19]:
vectorizer = TfidfVectorizer()

X_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Create Models

In [70]:
# svc = SVC(random_state=42)
lg = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
nb = MultinomialNB()
# xg = xgb.XGBRFClassifier(random_state=42) 

# Gridsearch Models

In [38]:
def make_gridsearch_model(name, model, params={}):
        return {
            "name": name,
            "model": model,
            "params": params
        }, 

In [83]:
def run_gridsearch(data, estimators, cv=3):

    for estimator in estimators:
        estimator = estimator[0]
        grid = GridSearchCV(estimator=estimator["model"], 
                            param_grid=estimator['params'], 
                            cv=cv, n_jobs=-1, scoring="balanced_accuracy")

        grid.fit(data["X_train"], data["y_train"])

        score = grid.best_estimator_.score(data["X_test"], data["y_test"])
        print("Best Params:", grid.best_params_)
        print(f"{estimator['name']}:\t{score}")

In [90]:
estimators = [
#     make_gridsearch_model("svc", svc),
#     make_gridsearch_model("lg", lg),
    make_gridsearch_model("rf", rf, 
                          {
#                               "max_depth": [x for x in range(10, 51)],
                              "n_estimators": [x for x in range(10, 100, 10)]
                          }),
#     make_gridsearch_model("nb", nb),
]

data = {
    "X_train": X_vectorized,
    "X_test": X_test_vectorized,
    "y_train": y_train,
    "y_test": y_test
}

run_gridsearch(data, estimators=estimators)

Best Params: {'n_estimators': 90}
rf:	0.7909585876918


# Check OneVsRest Performance on Models

In [68]:
ovr = OneVsRestClassifier(estimator=lg)

ovr.fit(X_vectorized, y_train)

ovr.score(X_test_vectorized, y_test)

/home/qdizon/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8120772149810263